# Model regresji: przewidywanie `Life expectancy`

Cel: zbudować i ocenić model regresji przewidujący oczekiwaną długość życia (*Life expectancy*) na podstawie zmiennych zdrowotnych, ekonomicznych i społecznych z pliku CSV.

Notebook prowadzi krok po kroku:
1) wczytanie danych, 2) kontrola braków i typów, 3) przygotowanie danych (uzupełnianie braków, standaryzacja, kodowanie `Status`), 4) dobór cech, 5) modele liniowe + regularyzacja (Ridge), 6) walidacja i analiza błędów, 7) interpretacja (permutacja), 8) zapis modelu.

Założenie (domyślne): **nie używamy `Country` jako cechy**, żeby uniknąć “identyfikowania kraju” i przeuczenia; zostawiamy `Year`, `Status` i pozostałe zmienne numeryczne.

In [ ]:
# 1) Ustawienia środowiska i import bibliotek
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge

# Ustawienia czytelności
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

sns.set_theme(style="whitegrid")

print("OK: biblioteki załadowane")
print("Wskazówka: jeśli brakuje scikit-learn, doinstaluj: pip install scikit-learn")

## 2) Wczytanie CSV i szybka walidacja danych

Uwaga: w oryginalnym zbiorze zdarzają się spacje na końcu nazw kolumn (np. `Life expectancy `). Poniżej **czyścimy nazwy kolumn** przez `.str.strip()` i dopiero potem wybieramy target.

In [ ]:
# 2) Wczytanie CSV i szybka walidacja danych

CSV_PATH = Path("Life Expectancy Data.csv")  # plik jest w katalogu projektu

if not CSV_PATH.exists():
    raise FileNotFoundError(f"Nie znaleziono pliku: {CSV_PATH.resolve()}")

df = pd.read_csv(CSV_PATH)

# standaryzacja nazw kolumn (w zbiorze zdarzają się spacje na końcu)
df.columns = df.columns.str.strip()

print("Wczytano df:")
print("- shape:", df.shape)
display(df.head())

# szybka sanity-check
print("\nKolumny:", list(df.columns))
print("\nPrzykładowe kraje:", df["Country"].dropna().unique()[:5])
print("Zakres lat:", df["Year"].min(), "-", df["Year"].max())

## 3) Podgląd struktury: typy kolumn, braki danych, duplikaty

W tym kroku:
- sprawdzamy typy kolumn,
- liczymy braki danych (procentowo),
- sprawdzamy duplikaty,
- robimy prosty wykres procentu braków.

In [ ]:
# 3) Typy, braki, duplikaty

display(df.info())

missing_frac = df.isna().mean().sort_values(ascending=False)
missing_tbl = (missing_frac * 100).round(2).to_frame("missing_%")
missing_tbl["missing_count"] = df.isna().sum()
missing_tbl = missing_tbl[missing_tbl["missing_count"] > 0]

print("\nKolumny z brakami (top 15):")
display(missing_tbl.head(15))

dup_count = df.duplicated().sum()
print(f"\nDuplikaty wierszy: {dup_count}")

cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("\nKolumny kategoryczne:", cat_cols)
for c in cat_cols:
    print(f"- {c}: {df[c].nunique(dropna=True)} unikalnych wartości")

# wykres procentu braków
if len(missing_tbl) > 0:
    plt.figure(figsize=(10, 5))
    missing_tbl["missing_%"].sort_values(ascending=True).plot(kind="barh")
    plt.title("Procent braków danych w kolumnach")
    plt.xlabel("% braków")
    plt.tight_layout()
    plt.show()

## Braki danych: co z nimi robimy?

W EDA wyszło, że usunięcie wierszy z brakami wycina ~44% danych i może biasować próbę (kraje z lepszą sprawozdawczością). Dlatego tutaj:

- **Nie usuwamy wierszy** z brakami w cechach (poza brakami w samym `y`).
- Uzupełniamy braki w danych w prosty sposób:
  - kolumny liczbowe → **mediana**,
  - `Status` → **najczęstsza wartość** + zamiana na kolumny 0/1 (One-Hot).

Opcjonalnie (zgodnie z narracją EDA o "wartościach z lat poprzednich"): można zrobić uzupełnianie **w obrębie kraju po czasie** (forward-fill/back-fill). To ma sens, ale trzeba uważać na wyciek informacji; jeśli chcesz, zrobimy to jako osobny eksperyment (train i test osobno).

In [ ]:
def country_time_impute(frame: pd.DataFrame, numeric_columns: list[str]) -> pd.DataFrame:
    out = frame.copy()
    if "Country" not in out.columns or "Year" not in out.columns:
        return out

    out = out.sort_values(["Country", "Year"]).reset_index(drop=True)

    # forward-fill i backward-fill w obrębie kraju
    out[numeric_columns] = (
        out.groupby("Country", as_index=False)[numeric_columns]
           .apply(lambda g: g.ffill().bfill())
           .reset_index(drop=True)
    )

    return out

print("OK: helper country_time_impute() gotowy (opcjonalny)")

## 4) Wybór zmiennej celu (target) i podział na X/y

Zmienna celu: **`Life expectancy`**.

Zgodnie z EDA: **nie używamy `Country` jako cechy** (żeby nie robić modelu „rozpoznaj kraj”). `Country` zostaje tylko jako ewentualny klucz do analiz/eksperymentów z imputacją po czasie.

In [ ]:
# 4) Target oraz X/y

TARGET_COL = "Life expectancy"

if TARGET_COL not in df.columns:
    raise KeyError(f"Nie ma kolumny targetu '{TARGET_COL}'. Dostępne: {list(df.columns)}")

# y musi być liczbowe
y = pd.to_numeric(df[TARGET_COL], errors="coerce")

# X: wszystkie kolumny poza targetem, ale bez Country
DROP_COLS = ["Country"]
X = df.drop(columns=[TARGET_COL] + [c for c in DROP_COLS if c in df.columns])

print("X shape:", X.shape)
print("y shape:", y.shape)

# szybki podgląd
print("\nX dtypes (top 10):")
print(X.dtypes.head(10))
print("\ny missing %:", round(y.isna().mean() * 100, 2))

## 5) Podział train/test (z kontrolą losowości)

Żeby porównywać modele uczciwie, robimy stały podział `train_test_split`.

Opcjonalnie: przy mocno skośnym rozkładzie targetu można robić pseudo-stratyfikację po binach `y` (tu robimy to „best-effort”: jeśli się nie da, wracamy do zwykłego split).

In [ ]:
# 5) train/test split

# Best-effort stratyfikacja po binach y (jeśli się da)
stratify_labels = None
try:
    y_bins = pd.qcut(y, q=10, duplicates="drop")
    if y_bins.nunique() >= 2 and y_bins.notna().all():
        stratify_labels = y_bins
except Exception:
    stratify_labels = None

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=stratify_labels,
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("y_train missing %:", round(y_train.isna().mean() * 100, 2))
print("y_test missing %:", round(y_test.isna().mean() * 100, 2))

# Jeśli target ma braki, usuwamy je TYLKO z y i odpowiadających wierszy X w danym zbiorze.
train_mask = y_train.notna()
test_mask = y_test.notna()
X_train = X_train.loc[train_mask]
y_train = y_train.loc[train_mask]
X_test = X_test.loc[test_mask]
y_test = y_test.loc[test_mask]

print("Po usunięciu braków w y:")
print("Train:", X_train.shape, "Test:", X_test.shape)

## 6) Przygotowanie danych: uzupełnianie braków + standaryzacja + kodowanie `Status`

W tym kroku przygotowujemy dane do uczenia modelu:
- kolumny liczbowe: uzupełnienie braków (mediana) i standaryzacja,
- `Status`: uzupełnienie braków (najczęstsza wartość) i kodowanie One-Hot.

Ważne: transformacje „uczymy” tylko na zbiorze treningowym i stosujemy je do zbioru testowego, żeby nie mieszać informacji z testu do treningu.

In [ ]:
# 6) Przygotowanie danych (ręczne fit/transform)

# Identyfikacja kolumn
numeric_features = X_train.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object"]).columns.tolist()

# W tym zbiorze sensownie zostawić tylko Status jako cechę kategoryczną
categorical_features = [c for c in categorical_features if c == "Status"]

print("Numeryczne:", numeric_features)
print("Kategoryczne:", categorical_features)

# Obiekty przygotowania danych
num_imputer = SimpleImputer(strategy="median")
scaler = StandardScaler()

cat_imputer = SimpleImputer(strategy="most_frequent")

# OneHotEncoder: kompatybilność między wersjami sklearn (sparse vs sparse_output)
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

def fit_preprocess(X_train: pd.DataFrame) -> dict:
    """Dopasuj obiekty przygotowania danych na zbiorze treningowym."""
    bundle = {}

    # numeryczne
    bundle["numeric_features"] = numeric_features
    bundle["num_imputer"] = num_imputer.fit(X_train[numeric_features])
    X_num = bundle["num_imputer"].transform(X_train[numeric_features])
    bundle["scaler"] = scaler.fit(X_num)

    # kategoryczne (Status)
    bundle["categorical_features"] = categorical_features
    if len(categorical_features) > 0:
        bundle["cat_imputer"] = cat_imputer.fit(X_train[categorical_features])
        X_cat = bundle["cat_imputer"].transform(X_train[categorical_features])
        bundle["ohe"] = ohe.fit(X_cat)
    else:
        bundle["cat_imputer"] = None
        bundle["ohe"] = None

    # nazwy cech po transformacji
    num_names = np.array(numeric_features, dtype=object)
    if bundle["ohe"] is not None:
        try:
            cat_names = bundle["ohe"].get_feature_names_out(categorical_features).astype(object)
        except Exception:
            cat_names = np.array([], dtype=object)
    else:
        cat_names = np.array([], dtype=object)
    bundle["feature_names"] = np.concatenate([num_names, cat_names])

    return bundle

def transform_preprocess(bundle: dict, X: pd.DataFrame) -> np.ndarray:
    """Zastosuj przygotowanie danych: imputacja + skalowanie + OneHot."""
    # numeryczne
    X_num = bundle["num_imputer"].transform(X[bundle["numeric_features"]])
    X_num = bundle["scaler"].transform(X_num)

    # kategoryczne
    if bundle.get("ohe") is not None and len(bundle.get("categorical_features", [])) > 0:
        X_cat = bundle["cat_imputer"].transform(X[bundle["categorical_features"]])
        X_cat = bundle["ohe"].transform(X_cat)
    else:
        X_cat = np.empty((len(X), 0))

    X_t = np.hstack([X_num, X_cat])

    # (opcjonalnie) dobór cech
    if "selector" in bundle:
        X_t = bundle["selector"].transform(X_t)

    return X_t

preprocess_bundle = fit_preprocess(X_train)
X_train_t = transform_preprocess(preprocess_bundle, X_train)
X_test_t = transform_preprocess(preprocess_bundle, X_test)

print("OK: przygotowanie danych gotowe")
print("- X_train_t:", X_train_t.shape)
print("- X_test_t:", X_test_t.shape)

## 7) Dobór cech: SelectKBest (f_regression)

Wymaganie projektu obejmuje dobór cech. Ponieważ po przygotowaniu danych mamy już macierz cech (`X_train_t`), wykonujemy dobór cech wyłącznie na zbiorze treningowym i stosujemy go na teście.

Metoda: `SelectKBest` z testem `f_regression` (univariate) – wybiera cechy o najsilniejszym związku liniowym z targetem.

In [ ]:
# 7) Dobór cech – SelectKBest
from sklearn.feature_selection import SelectKBest, f_regression

# Ustal liczbę wybieranych cech (prosty, rozsądny wybór do raportu)
k = min(20, X_train_t.shape[1])
selector = SelectKBest(score_func=f_regression, k=k)
selector.fit(X_train_t, y_train)

# Zastosuj dobór cech
X_train_t_fs = selector.transform(X_train_t)
X_test_t_fs = selector.transform(X_test_t)

selected_mask = selector.get_support()
selected_feature_names = preprocess_bundle["feature_names"][selected_mask]

# Podłączamy selector do bundle (żeby zapisany model dało się używać)
preprocess_bundle["selector"] = selector
preprocess_bundle["feature_names"] = selected_feature_names

# Dla dalszych kroków pracujemy już na macierzy po doborze cech
X_train_t = X_train_t_fs
X_test_t = X_test_t_fs

print("OK: dobór cech")
print("- k:", k)
print("- X_train_t:", X_train_t.shape)
print("- X_test_t:", X_test_t.shape)
display(pd.DataFrame({"feature": selected_feature_names}).head(20))

## 8) Funkcja ewaluacji (MAE / RMSE / $R^2$)

Dalej trenujemy modele na danych po preprocessingu (`X_train_t`, `X_test_t`).

In [ ]:
# 8) Ewaluacja – bez Pipeline

def evaluate_estimator(model, X_train_t, X_test_t, y_train, y_test, name: str) -> dict:
    model.fit(X_train_t, y_train)
    y_pred = model.predict(X_test_t)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    return {
        "model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
    }

results = []
print("OK: evaluate_estimator() gotowa")

## 8) Modele: dlaczego takie?

Wybór modeli opiera się na wnioskach z EDA:

- **Zależności są w dużej mierze liniowe** (korelacje + wykresy punktowe), więc startujemy od **regresji liniowej**.
- Jest sporo **współzależnych cech** (np. szczepienia korelują), co może destabilizować współczynniki w czystej regresji liniowej.
- Dlatego jako model „docelowy” bierzemy **Ridge Regression (L2)** – nadal liniowy, ale bardziej stabilny.

Nie używamy `Pipeline`: preprocessing robimy osobno (fit na train → transform train/test), a modele trenujemy na macierzy cech po transformacji.

In [ ]:
# 8) LinearRegression + Ridge (z tuningiem) – bez sklearn.Pipeline

# LinearRegression
lin_model = LinearRegression()
results.append(evaluate_estimator(lin_model, X_train_t, X_test_t, y_train, y_test, "LinearRegression"))

# Ridge z GridSearchCV (na danych po preprocessingu)
ridge = Ridge(random_state=RANDOM_STATE)
param_grid = {"alpha": np.logspace(-3, 3, 13)}

ridge_search = GridSearchCV(
    ridge,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1,
 )

ridge_search.fit(X_train_t, y_train)
print("Ridge best params:", ridge_search.best_params_)

ridge_best = ridge_search.best_estimator_
results.append(evaluate_estimator(ridge_best, X_train_t, X_test_t, y_train, y_test, "Ridge(best alpha)"))

results_df = pd.DataFrame(results).sort_values("RMSE")
display(results_df)

## 10) Walidacja krzyżowa i raport metryk

Walidacja krzyżowa (np. 5-fold) pozwala sprawdzić stabilność wyniku.

$$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^2}$$

In [ ]:
# 11) Cross-validation dla najlepszego modelu (wg RMSE na teście) – bez pipeline

best_name = results_df.iloc[0]["model"]
print("Najlepszy wg RMSE (test):", best_name)

if best_name.startswith("Ridge"):
    best_model = ridge_best
else:
    best_model = lin_model

scoring = {
    "MAE": "neg_mean_absolute_error",
    "RMSE": "neg_root_mean_squared_error",
    "R2": "r2",
}

cv = cross_validate(
    best_model,
    X_train_t,
    y_train,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False,
 )

cv_summary = pd.DataFrame(
    {
        "MAE": -cv["test_MAE"],
        "RMSE": -cv["test_RMSE"],
        "R2": cv["test_R2"],
    },
)

print("\nCV (5-fold) – średnia ± odch. std:")
for metric in ["MAE", "RMSE", "R2"]:
    print(f"- {metric}: {cv_summary[metric].mean():.4f} ± {cv_summary[metric].std():.4f}")

display(cv_summary)

## 11) Analiza błędów: $y_{true}$ vs $y_{pred}$ i reszty

Wykresy pozwalają szybko sprawdzić:
- czy błąd rośnie dla pewnych zakresów (heteroscedastyczność),
- czy są obserwacje odstające,
- czy model ma bias (systematyczne przeszacowanie/niedoszacowanie).

In [ ]:
# 11) Wykresy błędów na teście (bez pipeline)

best_model.fit(X_train_t, y_train)
y_pred = best_model.predict(X_test_t)
residuals = y_test - y_pred

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
min_v = min(y_test.min(), y_pred.min())
max_v = max(y_test.max(), y_pred.max())
plt.plot([min_v, max_v], [min_v, max_v], "r--", linewidth=2)
plt.xlabel("y_true")
plt.ylabel("y_pred")
plt.title("Predykcja vs rzeczywiste (test)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Rozkład reszt (y_true - y_pred)")
plt.xlabel("reszta")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(0, color="red", linestyle="--")
plt.title("Reszty vs y_pred")
plt.xlabel("y_pred")
plt.ylabel("reszta")
plt.tight_layout()
plt.show()

print("Reszty: średnia=", float(np.mean(residuals)), "std=", float(np.std(residuals)))

## 12) Ważność cech (Permutation Importance)

Ważność cech liczymy przez permutację na zbiorze testowym:
- mieszamy jedną kolumnę w `X_test_t`,
- patrzymy, jak bardzo pogarsza się wynik (tu: RMSE).

To działa dla modeli liniowych i nieliniowych.

In [ ]:
# 12) Permutation importance (bez pipeline)
from sklearn.inspection import permutation_importance

# dopasuj najlepszy model na train
best_model.fit(X_train_t, y_train)

# nazwy cech po preprocessingu (z naszego "ręcznego" preprocessu)
feature_names = np.array(preprocess_bundle.get("feature_names", []), dtype=object)

perm = permutation_importance(
    best_model,
    X_test_t,
    y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring="neg_root_mean_squared_error",
)

importances = pd.DataFrame(
    {
        "feature": feature_names,
        "importance_mean": perm.importances_mean,
        "importance_std": perm.importances_std,
    }
).sort_values("importance_mean", ascending=False)

display(importances.head(20))

plt.figure(figsize=(10, 6))
topk = importances.head(15).iloc[::-1]
plt.barh(topk["feature"], topk["importance_mean"], xerr=topk["importance_std"])
plt.title("Permutation importance (top 15) – spadek jakości po permutacji cechy")
plt.xlabel("Zmiana (neg_RMSE) – im większa wartość bezwzględna, tym ważniejsza cecha")
plt.tight_layout()
plt.show()

## 14) Sprawdzenie uogólniania: podział po krajach (Country-holdout)

W EDA dane są w układzie **kraj–rok**. Przy losowym podziale wierszy ten sam kraj trafia do train i test, co często **zawyża wynik**.

Dlatego robimy dodatkowy test: trenujemy na części krajów i testujemy na *innych* krajach. To mówi, jak model działa na „nowych” państwach.

In [ ]:
# 15) Country-holdout (bez Pipeline)

# przygotuj pełne X/y + grupy
_df = df.copy()
_df.columns = _df.columns.str.strip()

# target
_y_all = pd.to_numeric(_df[TARGET_COL], errors="coerce")

# cechy bez Country
_X_all = _df.drop(columns=[TARGET_COL, "Country"], errors="ignore")

# usuń braki w y
mask_all = _y_all.notna()
_X_all = _X_all.loc[mask_all].reset_index(drop=True)
_y_all = _y_all.loc[mask_all].reset_index(drop=True)
_groups = _df.loc[mask_all, "Country"].reset_index(drop=True)

countries = _groups.unique()
rng = np.random.RandomState(RANDOM_STATE)
rng.shuffle(countries)

cut = int(0.8 * len(countries))
train_countries = set(countries[:cut])

is_train = _groups.isin(train_countries)

X_train_c = _X_all.loc[is_train].reset_index(drop=True)
y_train_c = _y_all.loc[is_train].reset_index(drop=True)
X_test_c = _X_all.loc[~is_train].reset_index(drop=True)
y_test_c = _y_all.loc[~is_train].reset_index(drop=True)

print("Country-holdout:")
print("- train rows:", X_train_c.shape, "test rows:", X_test_c.shape)
print("- train countries:", len(train_countries), "test countries:", len(countries) - len(train_countries))

# dopasuj preprocess na train_c
numeric_features_c = X_train_c.select_dtypes(include=["number"]).columns.tolist()
categorical_features_c = [c for c in X_train_c.select_dtypes(include=["object"]).columns.tolist() if c == "Status"]

num_imputer_c = SimpleImputer(strategy="median")
scaler_c = StandardScaler()
cat_imputer_c = SimpleImputer(strategy="most_frequent")
try:
    ohe_c = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe_c = OneHotEncoder(handle_unknown="ignore", sparse=False)

# fit
Xtr_num = scaler_c.fit_transform(num_imputer_c.fit_transform(X_train_c[numeric_features_c]))
Xte_num = scaler_c.transform(num_imputer_c.transform(X_test_c[numeric_features_c]))

if len(categorical_features_c) > 0:
    Xtr_cat = ohe_c.fit_transform(cat_imputer_c.fit_transform(X_train_c[categorical_features_c]))
    Xte_cat = ohe_c.transform(cat_imputer_c.transform(X_test_c[categorical_features_c]))
else:
    Xtr_cat = np.empty((len(X_train_c), 0))
    Xte_cat = np.empty((len(X_test_c), 0))

X_train_ct = np.hstack([Xtr_num, Xtr_cat])
X_test_ct = np.hstack([Xte_num, Xte_cat])

# (opcjonalnie) ten sam dobór cech co w głównym eksperymencie
k_ct = min(k, X_train_ct.shape[1]) if "k" in globals() else min(20, X_train_ct.shape[1])
selector_ct = SelectKBest(score_func=f_regression, k=k_ct)
selector_ct.fit(X_train_ct, y_train_c)
X_train_ct = selector_ct.transform(X_train_ct)
X_test_ct = selector_ct.transform(X_test_ct)

# porównanie modeli na country-holdout (liniowe + regularizacja)
country_results = []
country_results.append(evaluate_estimator(LinearRegression(), X_train_ct, X_test_ct, y_train_c, y_test_c, "LinearRegression"))

ridge_c = GridSearchCV(
    Ridge(random_state=RANDOM_STATE),
    param_grid={"alpha": np.logspace(-3, 3, 13)},
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1,
)
ridge_c.fit(X_train_ct, y_train_c)
country_results.append(evaluate_estimator(ridge_c.best_estimator_, X_train_ct, X_test_ct, y_train_c, y_test_c, f"Ridge(alpha={ridge_c.best_params_['alpha']})"))

display(pd.DataFrame(country_results).sort_values("RMSE"))

## 15) Zapis: model + preprocess (bez Pipeline)

Zapisujemy bundle z:
- dopasowanymi transformatorami preprocessingu (`preprocess_bundle`),
- najlepszym modelem (`best_model`).

In [ ]:
# 15) Zapis bundle (model + preprocess) i przykład predykcji
import joblib

MODEL_PATH = Path("regression_model_and_preprocess.joblib")

bundle = {
    "preprocess_bundle": preprocess_bundle,
    "model": best_model,
}

joblib.dump(bundle, MODEL_PATH)
print("Zapisano bundle do:", MODEL_PATH.resolve())

loaded = joblib.load(MODEL_PATH)

sample = X_test.head(5)
sample_t = transform_preprocess(loaded["preprocess_bundle"], sample)
preds = loaded["model"].predict(sample_t)

preview = pd.DataFrame({
    "y_true": y_test.loc[sample.index].values,
    "y_pred": preds,
})
preview["abs_error"] = np.abs(preview["y_true"] - preview["y_pred"])
display(preview)

## 16) Wnioski do sprawozdania (regresja + dobór cech + regularyzacja)

- **Wyniki są spójne z EDA**: wykresy/korelacje sugerowały w dużej mierze liniowe zależności → modele liniowe osiągają wysokie $R^2$ i rozsądny błąd (RMSE).
- **Dobór cech (SelectKBest)** po preprocessingu pozwala ograniczyć liczbę zmiennych do tych najsilniej związanych liniowo z targetem; to upraszcza model i raport, bez zmiany założeń (model nadal jest liniowy).
- **Ridge (L2) jest preferowany jako model główny**: regularyzacja stabilizuje współczynniki przy współliniowości cech (co było widoczne w EDA) i zwykle poprawia uogólnianie.
- **Preprocessing jest kluczowy**: imputacja braków + skalowanie cech numerycznych + One-Hot dla `Status` umożliwia trening bez utraty dużej części danych (które wypadłyby przy `dropna`).
- **Test country-holdout** (inne kraje w teście) jest trudniejszy i często daje gorsze metryki niż losowy split, co jest typowe dla danych kraj–rok.